In [1]:
%load_ext autoreload
%autoreload 2

import logging
logging.basicConfig(level=logging.INFO)
from rexpand_pyutils_file import write_file
from utils.conversation_data import load_conversation_data
from utils.json import to_json_compatible
from models.context import Context,ConversationMessage
from nodes.orchestrator import orchestrate
from models.workflow import State
import json
from nodes.orchestrator_langgraph import orchestrate as orchestrate_langgraph

INFO:root:Read JSON data from ./input/categories.json
INFO:root:Read JSON data from ./input/categories.json
INFO:root:Read JSON data from ./input/categories.json
INFO:root:Read JSON data from ./input/categories.json


In [3]:
# Load JSON conversations from file
with open('./input/test1_conversation.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

messages_test = [
    ConversationMessage(
        id=item['id'],
        role=item['role'],
        name=item.get('name', ''),
        body_text=item['body_text'],
        attachments_text=item['attachments_text'],
        delivered_at=item['delivered_at']
    )
    for item in data
]


In [4]:
# Select a conversation to process
messages_end = 2
messages = messages_test[:messages_end]
context = Context(messages=messages)
write_file('./output/current_messages.json', to_json_compatible(messages))


INFO:root:Wrote JSON data to ./output/current_messages.json


Develop with Langgraph

In [11]:
# Fresh start. In this case, no human in the loop with topics suggested
state = State(context=context, auto_assign_actions=False, auto_assign_topics=False)
app = orchestrate_langgraph(state)
print(app.invoke(state))
# for output in app.stream(state):
#     # stream() yields dictionaries with output keyed by node name
#     for key, value in output.items():
#         print(f"Output from node '{key}':")
#         print("---")
#         print(value)
#     print("\n---\n")


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


{'step': 'todo: human action required', 'context': Context(messages=[ConversationMessage(id='06aebc21-ed8c-4c8e-89c0-35dabb049025', role='job seeker', name='Mingzhi Li', body_text="Hi Niraj,\nI'm Mingzhi. I'm eager to explore the new Software Developer opening at Oracle. Could we connect for more info or a referral?\nKind regards,\nMingzhi.", attachments_text='', delivered_at=1741098268349), ConversationMessage(id='56c4d155-5b08-4758-b44c-f5fe648bf4e1', role='referrer', name='Niraj P.', body_text="Hi, Mingzhi,\nSure, we can connect. \nPlease share your resume and we'll start from there. \nRegards\nNiraj", attachments_text='', delivered_at=1741098324075)], user_profile=None, referrer_profile=None), 'classified_category': ClassifierResult(category='agree_on_condition', confidence=0.95, reason='The referrer, Niraj, explicitly agrees to connect but requests the job seeker to share their resume first, indicating a conditional agreement to help.', referenced_message_ids=['56c4d155-5b08-4758-

Develop with business logic

In [ ]:
# Fresh start. In this case, no human in the loop with topics suggested
state = State(context=context, auto_assign_actions=False, auto_assign_topics=False)
state = orchestrate(state)

print(state.step)
print(state.classified_category)
print(state.summarized_actions)
print(state.fulfilled_actions)  # This is the selected actions after summarization
print(state.inferred_results)
print(state.suggested_topics)
print(state.selected_topics)
print(state.generated_reply_message)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


next: select topics
ClassifierResult(category='agree_on_condition', confidence=0.95, reason='The referrer, Niraj, agrees to connect but requests the job seeker to share their resume first, indicating a conditional agreement to help.', referenced_message_ids=['56c4d155-5b08-4758-b44c-f5fe648bf4e1'])
ActionSummarizerResult(actions=[Action(action='Provide the latest resume to Niraj P.', confidence=0.95, reason='Niraj agreed to connect and requested the resume to proceed further.')])
ActionSummarizerResult(actions=[Action(action='Provide the latest resume to Niraj P.', confidence=0.95, reason='Niraj agreed to connect and requested the resume to proceed further.')])
InferenceResult(inferences=[Inference(inference=True, confidence=0.9, reason="The referrer, Niraj, responded positively to the job seeker's request by agreeing to connect and asking for the resume to start the referral process. This indicates a willingness to refer, contingent on receiving the resume.")])
TopicSuggesterResult(to